In [ ]:

# Input data book. Took Chapter 1 from this book to train the model.
# https://edu.anarcho-copy.org/Against%20Security%20-%20Self%20Security/Dafydd%20Stuttard,%20Marcus%20Pinto%20-%20The%20web%20application%20hacker's%20handbook_%20finding%20and%20exploiting%20security%20flaws-Wiley%20(2011).pdf

In [2]:
!pip install langchain faiss-cpu pypdf transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 29.4 MB/s eta 0:00:00


In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
loader = DirectoryLoader("/content/", glob="*.pdf", loader_cls=PyPDFLoader)

In [5]:
data = loader.load()

In [6]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[3].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')

You have 16 document(s) in your data
There are 2016 characters in your sample document
Here is a sample: Stuttard   c01.indd   V2 - 07/07/2011 Page 1
1CHAPTER  
1
Web Application (In)security
There is no doubt that web application security is a current and newsworthy 
subject. For all concerned, the stak


In [7]:
print(data[5].page_content)

Stuttard   c01.indd   V2 - 07/07/2011 Page 6
6 Chapter 1 n Web Application (In)security
input controls that are immediately familiar to users, avoiding the need 
to learn how each individual application functions. Client-side scripting 
enables applications to push part of their processing to the client side, and 
browsers’ capabilities can be extended in arbitrary ways using browser 
extension technologies where necessary.
 nThe core technologies and languages used to develop web applications are 
relatively simple. A wide range of platforms and development tools are 
available to facilitate the development of powerful applications by relative 
beginners, and a large quantity of open source code and other resources 
is available for incorporation into custom-built applications.
Web Application Security
As with any new class of technology, web applications have brought with them 
a new range of security vulnerabilities. The set of most commonly encountered 
defects has evolved somewhat

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [9]:
print (f'Now you have {len(texts)} documents')

Now you have 108 documents


In [10]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00


In [13]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs= { 'device' : 'cpu' })

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
db = FAISS.from_documents(texts, embeddings)

In [15]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'# go for a smaller model if you dont have the VRAM
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1000
)

local_llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize on the basis of data found in these retrieved docs: {docs}"
)


# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | local_llm | StrOutputParser()

# Run
question = "What is the future of Web Application Security? Give details."
docs = db.similarity_search(question)
chain.invoke(docs)

'Stuttard c01.indd V2 - 07/07/2011 Page 1 1CHAPTER 1 Web Application (In)security There is no doubt that web application security is a current and newsworthy subject. For all concerned, the stakes are high: for businesses that derive increa- ing revenue from Internet commerce, for users who trust web applications with is available for incorporation into custom-built applications. Web Application Security As with any new class of technology, web applications have brought with them a new range of security vulnerabilities.'